In [106]:
import xlrd, xlwt
import requests
from xlwt import Workbook

In [107]:
excelPath = ("FullDatabase04082020.xlsx")

In [108]:
workbook = xlrd.open_workbook(excelPath) # read workbook
sheet = workbook.sheet_by_index(0)

In [109]:
firstRow = []
for i in range(sheet.ncols):
    firstRow.append(sheet.cell_value(0, i))

In [110]:
Title, Authors, Journal, DOI = [], [], [], []
excelRow = 0;
for i in range(0, sheet.nrows):
    if sheet.cell_type(i, 10) != xlrd.XL_CELL_EMPTY:
        # filter DOI to be usable
        DOIValue = sheet.cell_value(i, 10)
        DOIOnly = DOIValue.replace("https://doi.org/", "")
        DOIOnly = DOIOnly.replace("http://dx.doi.org/", "")
        DOIOnly = DOIOnly.replace("doi:", "")
        DOIOnly = DOIOnly.split(' ', 1)[0]
        if DOIOnly.startswith("DOI") or DOIOnly.startswith("10"):
            DOI.append(DOIOnly)
            Title.append(sheet.cell_value(i, 0))
            Authors.append(sheet.cell_value(i, 1))
            Journal.append(sheet.cell_value(i, 5))    

In [111]:
wb = Workbook() # write workbook 
filteredSheetForWriting = wb.add_sheet("Filtered Sheet")

In [112]:
# write filtered data
for i in range(len(DOI)):
    filteredSheetForWriting.write(i, 0, Title[i])
    filteredSheetForWriting.write(i, 1, Authors[i])
    filteredSheetForWriting.write(i, 2, Journal[i])
    filteredSheetForWriting.write(i, 3, DOI[i])

In [113]:
wb.save("FilteredDatabase.xls")

PermissionError: [Errno 13] Permission denied: 'FilteredDatabase.xls'

In [ ]:
# Identify Author's affiliation
# https://api.crossref.org/v1/works/
filteredWorkbook = xlrd.open_workbook("FilteredDatabase.xls")
filteredSheet = filteredWorkbook.sheet_by_index(0)
baseURL = "https://api.crossref.org/v1/works/"

In [ ]:
URLS = []
for i in range(1, filteredSheet.nrows):
    URLS.append(baseURL + filteredSheet.cell_value(i,3))
#r = requests.get(baseURL)
#print(r.json())

In [ ]:
pageInfo =[]
for i in range(20):
    page = requests.get(URLS[i])
    pageJson = page.json()
    pageInfo.append(pageJson)
    #filteredSheetForWriting.write(i, 4, pageInfo[i])

In [ ]:
for i in range(len(pageInfo)):
    filteredSheetForWriting.write(i, 5, str(pageInfo[i].get("message").get("author")[0].get("affiliation")))
    print(pageInfo[i].get("message").get("author")[0].get("affiliation"))

In [ ]:
wb.save("FilteredDatabase.xls")